In [1]:
import openai
import textwrap as _tw
from credentials import api_key
openai.api_key = api_key

MODEL = "gpt-3.5-turbo"

MODEL4 = 'gpt-4'

Verify working


In [32]:


initial_messages = [{"role":"system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": "Why is the sky blue?"},
                    {"role": "assistant", "content" : "The sky is not really blue, but "}]
completion = openai.ChatCompletion.create(
    model=MODEL,
    messages= initial_messages)
print(_tw.fill(completion.choices[0].message.content, width=100))

appears that way because of the way our atmosphere scatters sunlight. When light from the sun enters
our atmosphere, it encounters gases and particles in the air that scatter the light in all different
directions. This happens most strongly with shorter, blue wavelengths of light, which is why the sky
looks blue to us.


## Goals

* Generate a story and then interact with it by interjecting changes to it. 

*  TODO: Allow user to save/load a story.

* TODO: Add a method to create a story summary for continuation. (partially done, and seems to work). But Ideally we should have another class that manages the various 'chapters' and can combine them (without the 'story so far' part.)  

* BUG: Summarizing a story that was based on a summary causes the story have multiple "story so far" sections.

In [35]:
initial_messages = [{"role":"system", "content": "You are a helpful editor."},
                    {"role": "user", "content": "Finish the final sentance in this text: The car was very fast! However, we"} ]
completion = openai.ChatCompletion.create(
    model=MODEL,
    messages= initial_messages)
print(_tw.fill(completion.choices[0].message.content, width=100))

were unable to appreciate its full potential due to the heavy traffic on the road.


In [2]:
class gpt_story:
    def __init__(self, topic = "anything", model = MODEL, max_tokens = 100):
        self.model = model
        self.topic = topic
        self.max_tokens = max_tokens
        self.based_on_summary = False
        self.messages = [{"role":"system", "content": "You are a creative writer."},
                                 {"role": "user", "content": f"Write a story about {topic}."}]


    def query_gpt(self):
        completion = openai.ChatCompletion.create(
            model=self.model,
            messages= self.messages, max_tokens = self.max_tokens)
        self.messages = self.messages + [dict(completion.choices[0].message)]

    def tell_story(self):
        self.query_gpt()
        self.clean_up_last_message()
        return self
    
    # This serves to make sure the last message is a complete sentence. Currently if the last sentence is complete
    # it still adds another. This is a bug that should be fixed but not critical.
    def clean_up_last_message(self):
        last_message = self.messages[-1]['content']
        cleanup_messages = [{"role":"system", "content": "You are a helpful editor."},
                    {"role": "user", "content": "Finish the final sentence in this text if needed: " + last_message} ]
        completion = openai.ChatCompletion.create(model=MODEL, messages= cleanup_messages)
        self.messages[-1]['content'] = last_message + " " + completion.choices[0].message.content
        return self

        
    def continue_with(self, message):
        message_dict = {'role':'assistant', 'content' : message}
        self.messages = self.messages +  [message_dict]
        self.tell_story()
        return self


    def summarize(self): 
        story_so_far = ' '.join([message['content'] for message in self.messages[2:]])
        summary = openai.Completion.create(
            model = "text-davinci-003",
            prompt = story_so_far + "tl;dr",
            max_tokens = 100
        )
        # Generate a new story starting with the summary
        new_story = gpt_story(topic=self.topic, model=self.model, max_tokens=self.max_tokens)
        new_story.messages += [{"role": "user", "content": "The story so far: "+summary.choices[0].text}]
        new_story.based_on_summary = True
        return new_story
            

    def __repr__(self):
        if self.based_on_summary:
            return _tw.fill(' '.join([message['content'] for message in self.messages[3:]]), width=120)
        return _tw.fill(' '.join([message['content'] for message in self.messages[2:]]), width=120)
    
    def __len__(self):
        return len(self.messages)
    
    def __getitem__(self, key):
        return self.messages[key]
    
    def __iter__(self):
        return iter(self.messages)
 

For demonstration purposes, we will use a short max token length of 60. This will allow us to see the story generation process in a reasonable amount of time.

In [3]:
test = gpt_story("sailing", max_tokens=60, model=MODEL)
test.tell_story()

The sun had just begun to rise, casting a soft orange glow across the ocean's surface. The morning breeze carried the
salty scent of the sea, and tiny waves gently crashed against the side of the boat. John, a seasoned sailor, adjusted
his sails and took a deep breath of the fresh ocean air, feeling grateful to be alive and witnessing the breathtaking
beauty of nature.

Ok, we can continue with the story generation process by just calling `tell_story` again and again:

In [4]:
test.tell_story()

The sun had just begun to rise, casting a soft orange glow across the ocean's surface. The morning breeze carried the
salty scent of the sea, and tiny waves gently crashed against the side of the boat. John, a seasoned sailor, adjusted
his sails and took a deep breath of the fresh ocean air, feeling grateful to be alive and witnessing the breathtaking
beauty of nature. John had been sailing since he was a teenager. He loved the freedom it gave him, and the feeling of
being one with the sea. Today, however, he felt a different kind of excitement. He was on his way to meet his childhood
friend, Jake, who had just bought a sailboat and they were planning to sail together for the first time in years.

But, if we want to continue the story in a different way, we can add our own text to the story and then continue from there:

In [5]:
test.continue_with("He had always dreamed of sailing a large double masted boat like the one Jake had aquired.")

The sun had just begun to rise, casting a soft orange glow across the ocean's surface. The morning breeze carried the
salty scent of the sea, and tiny waves gently crashed against the side of the boat. John, a seasoned sailor, adjusted
his sails and took a deep breath of the fresh ocean air, feeling grateful to be alive and witnessing the breathtaking
beauty of nature. John had been sailing since he was a teenager. He loved the freedom it gave him, and the feeling of
being one with the sea. Today, however, he felt a different kind of excitement. He was on his way to meet his childhood
friend, Jake, who had just bought a sailboat and they were planning to sail together for the first time in years. He had
always dreamed of sailing a large double masted boat like the one Jake had aquired. As John approached the shore where
Jake was waiting, he could see the excitement in his friend's eyes. Together, they hoisted the sails and set out on the
vast expanse of the ocean.  They sailed for hour

Finally, if the story starts getting too long, we can summarize it and then continue from there. This will not include the previous parts however.

In [6]:
chapter2 = test.summarize()

In [7]:
chapter2.messages

[{'role': 'system', 'content': 'You are a creative writer.'},
 {'role': 'user', 'content': 'Write a story about sailing.'},
 {'role': 'user',
  'content': "The story so far: \n\nJohn and Jake went sailing together on a beautiful day. John had dreamed of sailing a double masted boat and was excited to finally see Jake's new sailboat. They spent hours at sea, taking turns steering and taking in the peaceful surroundings. It was a perfect day for sailing and a great experience for them both."}]

In [10]:
chapter2.tell_story()

As the sun began to set, they decided to anchor at a nearby island and set up camp for the night. They built a fire and
grilled fresh fish that John had caught earlier that day. With full stomachs, they sat by the fire and talked about
their lives, dreams, and adventures.   The sound of waves and the crackling fire lured them into a peaceful slumber
under the starry sky.